# Reproducing Figures 2, 3, and A3: Multiscale Entropy in Real-World Networks

This notebook reproduces **Figures 2 and 3** from Section 4.2 and **Figure A3** from the Appendix, which analyze multiscale entropy behavior across real-world networks from six domains: Biological, Social, Economic, Technological, Transportation, and Informational.

## Overview

We apply the multiscale entropy framework to 439 undirected networks from the ICON dataset, stratified by:

1. **Domain**: Biological, Social, Economic, Technological, Transportation, Informational
2. **Size**: Small (0-200 nodes), Medium (200-600 nodes), Large (600+ nodes)

The analysis reveals three distinct entropy trajectory patterns:
- **Stable behavior**: Biological, Transport, and Informational networks
- **Increasing behavior**: Economic and Technological networks  
- **Hybrid behavior**: Social networks


## Imports and Setup

In [ ]:
pip install pandas==1.5.3
pip install sklearn

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_1441/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import os
import sys

##get current file directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
##get the parent directory
parent_dir = os.path.dirname(current_dir)
# Add the src directory to sys.path
src_dir = os.path.join(parent_dir, '../')
sys.path.append(src_dir)

from algorithm.calculo_entropia import *
import algorithm.calculo_entropia

from algorithm.coarsening_utils import *
import algorithm.graph_utils
import algorithm.coarsening_utils as cu
from algorithm.coarsening_utils import plot_coarsening_vertical

import numpy as np
import scipy as sp

import matplotlib
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import networkx as nx
import pygsp as gsp
from pygsp import graphs
gsp.plotting.BACKEND = 'matplotlib'

import pickle

def save_graphs(graph_dict, filename):
    """
    Save the dictionary of graphs to a file.
    """
    # Convert PyGSP graphs to NetworkX graphs for easier serialization
    nx_graph_dict = {
        size: [nx.from_scipy_sparse_array(g.W) for g in graphs]
        for size, graphs in graph_dict.items()
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(nx_graph_dict, f)
    print(f"Graphs saved to {filename}")

def load_graphs(filename):
    """
    Load the dictionary of graphs from a file.
    """
    with open(filename, 'rb') as f:
        nx_graph_dict = pickle.load(f)
    print(f"Graphs loaded from {filename}")
    return nx_graph_dict

In [ ]:
import pandas as pd
print(pd.__version__)

1.5.3


In [ ]:
import pickle  
# load the data 
infile = open('./CommunityFitNet_updated.pickle','rb')  
df = pickle.load(infile) 

In [ ]:
# read edge lists for all networks
df_edgelists = df['edges_id'] # column 'edges_id' in dataframe df includes the edge list 
                              # for each network 
 
# extract the edge list for the first network 
edges_orig = df_edgelists.iloc[0] # a numpy array of edge list for original graph 


## Step 1: Data Loading and Preprocessing

We load the undirected networks from the ICON dataset and prepare them for multiscale analysis. The dataset has been pre-filtered to contain only undirected graphs compatible with the spectral reduction algorithm.

## Step 2: Multiscale Reduction and Entropy Calculation

For each network, we apply spectral coarsening at reduction levels of 100%, 80%, 60%, 40%, and 20% of the original size. At each level, we:

1. Apply the spectral coarsening algorithm with K=10 eigenvalues preserved
2. Convert the reduced graph to NetworkX format
3. Compute compression-based entropy using arithmetic encoding
4. Normalize against 10 Erdős-Rényi random graphs with matched size and density

This process generates a comprehensive JSON file (`graph_families_analysis.json`) containing:
- Graph metadata (domain, subdomain, node/edge types)
- Structural metrics at each reduction level (nodes, edges, average degree)
- Entropy measurements (raw, random baseline, normalized)

**Note**: This step is computationally intensive and may take several hours depending on the number of networks. Progress is tracked using tqdm.

In [ ]:
import pickle
import networkx as nx
import numpy as np
from pygsp import graphs
import json
from collections import defaultdict
from tqdm import tqdm  # For progress tracking
import pandas as pd

# Load the graph processing functions

import algorithm.calculo_entropia as ce

def create_networkx_graph(row):
    """Create a NetworkX graph from a row of the dataset."""
    is_directed = 'Directed' in row['graphProperties']
    G = nx.DiGraph() if is_directed else nx.Graph()
    
    nodes = np.array(row['nodes_id'])
    edges = np.array(row['edges_id'])
    
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
    return G

def calculate_graph_metrics(G_nx):
    """Calculate basic metrics for a graph."""
    return {
        'number_nodes': G_nx.number_of_nodes(),
        'number_edges': G_nx.number_of_edges(),
        'ave_degree': 2 * G_nx.number_of_edges() / G_nx.number_of_nodes()
    }

def calculate_entropies(G_nx):
    """Calculate entropy metrics using arithmetic encoding method."""
    # Get entropy metrics directly using the helper function
    entropy_metrics = cu.get_entropy_metadata_aritmethicEncoding(G_nx)
    
    return {
        'entropy_arithmetic': {
            'graph': entropy_metrics['Grafo'],
            'random': entropy_metrics['Grafo_r'],
            'normalized': entropy_metrics['Entropy Normalizado']
        }
    }

def process_graph_at_reduction(G_nx, reduction_percent):
    """Process a graph at a specific reduction percentage."""
    # Convert to PyGSP format
    W = nx.to_scipy_sparse_array(G_nx)
    G = graphs.Graph(W)
    
    # Calculate reduction ratio
    r = 1 - (reduction_percent/100)
    
    if reduction_percent == 100:
        # Original graph, no reduction needed
        G_reduced = G_nx
    else:
        # Perform coarsening
        try:
            C, Gc, Call, Gall = coarsen(G, K=10, r=r)
            G_reduced = nx.from_scipy_sparse_array(Gc.W)
        except Exception as e:
            print(f"Error in coarsening at {reduction_percent}%: {str(e)}")
            return None
    
    # Calculate metrics and entropies
    metrics = calculate_graph_metrics(G_reduced)
    entropies = calculate_entropies(G_reduced)
    
    # Combine all information
    return {
        'graph_portion': reduction_percent,
        **metrics,
        **entropies
    }


def create_graph_family_json(df):
    """Create the complete JSON structure for all graph families."""
    result = defaultdict(lambda: defaultdict(dict))
    
    # Process each row in the dataframe
    for _, row in tqdm(df.iterrows(), total=len(df)):
        family = row['networkDomain']
        if not isinstance(family, str):
            continue
            
        graph_data = {
            'Name': row['network_name'],
            'Subdomain': row['subDomain'],
            'Node_Type': row['nodeType'],
            'Edge_Type': row['edgeType'],
            'reductions': {}
        }
        
        # Create NetworkX graph
        G_nx = create_networkx_graph(row)
        
        # Process each reduction percentage
        for reduction in [100, 80, 60, 40, 20]:
            reduction_data = process_graph_at_reduction(G_nx, reduction)
            if reduction_data:
                graph_data['reductions'][str(reduction)] = reduction_data
        
        # Add to result
        result[family][row['network_name']] = graph_data
    
    return dict(result)

# Load the data
with open('undirected_networks.pkl', 'rb') as f:
    real_graphs = pickle.load(f)

# Process all graphs and create JSON
graph_families = create_graph_family_json(real_graphs)

# Save to JSON file
with open('graph_families_analysis.json', 'w') as f:
    json.dump(graph_families, f, indent=2)

# Example of accessing the data
print("Available network domains:", list(graph_families.keys()))
for domain in graph_families:
    print(f"\n{domain} networks:", list(graph_families[domain].keys()))

## Step 3: Case Study Visualization - Figure A3

To illustrate the diversity of entropy behaviors, we select 6 representative networks—one from each domain—and visualize their coarsening process across all reduction levels. This produces **Figure A3** in the Appendix.

### Selection Criteria

Networks are selected to maximize domain diversity while choosing medium-sized examples (median node count within each domain) for optimal visualization. If fewer than 6 domains are available, we select from different subdomains to ensure structural diversity.

### Visualization Details

For each network, we generate a horizontal panel showing:
- The graph structure at each reduction level (100%, 80%, 60%, 40%, 20%)
- Node and edge counts at each scale
- Visual preservation of key structural features

The visualization uses spring layout for node positioning and color-codes nodes by their contraction sets to show how the coarsening algorithm groups vertices.

In [ ]:
import pickle  
# load the data 
infile = open('./undirected_networks.pkl','rb')  
df = pickle.load(infile) 

# read edge lists for all networks
df_edgelists = df['edges_id'] # column 'edges_id' in dataframe df includes the edge list 
                              # for each network 
 
# extract the edge list for the first network 
edges_orig = df_edgelists.iloc[0] # a numpy array of edge list for original graph 

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

# After loading your df:
print(df.head())  # See first few rows
print(df.columns)  # See what columns we have
print(df.info()) 
print(len(df))  # Number of rows in the DataFrame

In [ ]:
# Simple analysis of 6 networks from different families

import numpy as np
import pygsp as gsp
import networkx as nx

# Parameters
reduction_levels = [0.2, 0.4, 0.6, 0.8]
method = "variation_neighborhood"
K = 5

# First, let's see what families we have
print("=== Available Network Families ===")
families = df.groupby(['networkDomain', 'subDomain']).agg({
    'network_index': 'count',
    'number_nodes': ['min', 'max'],
    'number_edges': ['min', 'max']
}).round(0)
print(families)
print()

# Select 6 networks - prioritize getting one from each DOMAIN first
print("=== Selecting Networks by Domain ===")
domains = df['networkDomain'].unique()
print(f"Available domains: {domains}")

selected_networks = []
used_domains = set()

# First pass: get one network from each domain
for domain in domains:
    domain_networks = df[df['networkDomain'] == domain]
    if len(domain_networks) > 0:
        # Pick a medium-sized network from this domain
        sorted_by_size = domain_networks.sort_values('number_nodes')
        median_idx = len(sorted_by_size) // 2
        selected_network = sorted_by_size.iloc[median_idx]
        selected_networks.append(selected_network)
        used_domains.add(domain)
        print(f"Selected from {domain}: {selected_network['title'][:50]}...")
        
        if len(selected_networks) >= 6:
            break

# Second pass: if we still need more networks, get from different subdomains
if len(selected_networks) < 6:
    print("\nNeed more networks, selecting from different subdomains...")
    
    # Get all unique domain-subdomain combinations we haven't used
    used_combinations = set()
    for net in selected_networks:
        used_combinations.add((net['networkDomain'], net['subDomain']))
    
    remaining_combinations = []
    for _, row in df.iterrows():
        combo = (row['networkDomain'], row['subDomain'])
        if combo not in used_combinations:
            remaining_combinations.append(row)
    
    # Sort by domain diversity, then by size
    remaining_df = pd.DataFrame(remaining_combinations)
    if len(remaining_df) > 0:
        for _, network in remaining_df.iterrows():
            selected_networks.append(network)
            print(f"Added from {network['networkDomain']}-{network['subDomain']}: {network['title'][:50]}...")
            if len(selected_networks) >= 6:
                break

# Print information for selected networks
print("=== Selected Networks Information ===")
for i, network in enumerate(selected_networks[:6], 1):
    print(f"Network {i}:")
    print(f"  Title: {network['title']}")
    print(f"  Description: {network['description']}")
    print(f"  Domain: {network['networkDomain']}")
    print(f"  Subdomain: {network['subDomain']}")
    print(f"  Nodes: {network['number_nodes']}")
    print(f"  Edges: {network['number_edges']}")
    print(f"  Average Degree: {network['ave_degree']:.2f}")
    print("-" * 60)

# Function to create PyGSP graph from edges
def create_graph_from_edges(edges_array):
    nx_graph = nx.Graph()
    nx_graph.add_edges_from(edges_array)
    adj_matrix = nx.adjacency_matrix(nx_graph)
    G = gsp.graphs.Graph(adj_matrix)
    # Add coordinates for visualization
    pos = nx.spring_layout(nx_graph, seed=42)
    coords = np.array([pos.get(i, [0, 0]) for i in range(G.N)])
    G.set_coordinates(coords)
    return G

# Visualize each network's coarsening
print("\n=== Visualizing Network Coarsening ===")
for i, network in enumerate(selected_networks[:6], 1):
    print(f"\nProcessing Network {i}: {network['title'][:50]}...")
    
    try:
        # Get edges for this network
        edges_orig = df[df['network_index'] == network['network_index']]['edges_id'].iloc[0]
        
        # Create PyGSP graph
        G = create_graph_from_edges(edges_orig)
        print(f"Created graph with {G.N} nodes, {G.Ne} edges")
        
        # Create visualization
        title = f"Network {i}: {network['title'][:40]}{'...' if len(network['title']) > 40 else ''}"
        fig = plot_reduction_levels_horizontal(
            G,
            reduction_levels=reduction_levels,
            method=method,
            K=K,
            size=3,
            alpha=0.7,
            title=title
        )
        plt.show()
        
    except Exception as e:
        print(f"Error with network {i}: {e}")
        continue

print("\nDone! ✓")

## Step 4: Cluster Analysis - Figure 4

To quantitatively validate the three distinct entropy trajectory patterns observed in Figures 2 and 3, we perform unsupervised clustering on the multiscale entropy profiles. This analysis produces **Figure 4** from Section 4.3.

### Clustering Methodology

Each network is represented as a 5-dimensional feature vector containing normalized compression entropy values at all reduction levels: [100%, 80%, 60%, 40%, 20%].

We apply K-means clustering with k=3 to identify the three behavioral regimes:

1. **Cluster 1 (Hybrid)**: Social networks showing stable entropy under moderate reduction, with sharp increases at aggressive compression
2. **Cluster 2 (Increasing)**: Economic and Technological networks exhibiting early structural degradation
3. **Cluster 3 (Stable)**: Biological, Transportation, and Informational networks maintaining consistent entropy across scales

### Dimensionality Reduction

To visualize the 5-dimensional entropy space, we apply Principal Component Analysis (PCA) to project data into 2D while preserving maximum variance. The resulting scatter plot shows:

- Each point represents a network, colored by domain
- Red "X" markers indicate cluster centroids
- Dashed gray circles illustrate cluster boundaries


In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from collections import defaultdict
from sklearn.decomposition import PCA

# Cargar los datos
with open("graph_families_analysis.json", "r") as file:
    data = json.load(file)

# Extraer los valores de entropía normalizada de length compression
graphs = []  # Lista de nombres de grafos
features = []  # Lista para almacenar los vectores de 5 dimensiones
families = []

for family, graphs_data in data.items():
    for graph_name, graph_info in graphs_data.items():
        if "reductions" in graph_info:
            reductions = graph_info["reductions"]
            if all(str(p) in reductions for p in ["100", "80", "60", "40", "20"]):
                entropy_values = [
                    reductions[str(p)]["entropy_arithmetic"]["normalized"]
                    for p in [100, 80, 60, 40, 20]
                ]
                features.append(entropy_values)
                graphs.append(graph_name)
                families.append(family)

# Convertir a numpy array
X = np.array(features)

# Estandarizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar KMeans con 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X_scaled)
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

# Visualizar los clusters usando PCA para reducción a 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
centers_pca = pca.transform(cluster_centers)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=families, palette="tab10", s=100, alpha=0.8)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c="red", marker="x", s=200, label="Cluster Center")

# Dibujar círculos alrededor de los clusters
for center in centers_pca:
    plt.gca().add_patch(plt.Circle(center, 0.5, color='gray', fill=False, linestyle='dashed'))

plt.title("Graph Clustering by Length Compression Normalized Entropy (KMeans, k=3)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend(title="Family Domain")
plt.show()

# Agrupar datos por cluster
cluster_composition = defaultdict(lambda: defaultdict(int))
for i, (family, graph_name) in enumerate(zip(families, graphs)):
    cluster_composition[labels[i]][family] += 1

# Mostrar la composición de los clusters
print("Composición de los clusters:\n")
for cluster, family_counts in sorted(cluster_composition.items()):
    print(f"Cluster {cluster}:")
    for family, count in family_counts.items():
        print(f"{family}: {count} grafos")
    print("")
